Ref: [LangChain - Custom Output Parsers](https://python.langchain.com/v0.1/docs/modules/model_io/output_parsers/custom/)

# Custom Output Parsers
In some situations you may want to implement a custom parser to structure the model output into a custom format.

There are two ways to implement a custom parser:

1. Using `RunnableLambda` or `RunnableGenerator` in LCEL -- we strongly recommend this for most use cases
2. By inherting from one of the base classes for out parsing -- this is the hard way of doing things

The difference between the two approaches are mostly superficial and are mainly in terms of which callbacks are triggered (e.g., `on_chain_start` vs. `on_parser_start`), and how a runnable lambda vs. a parser might be visualized in a tracing platform like LangSmith.

## Runnable Lambdas and Generators
The recommended way to parse is using runnable lambdas and runnable generators!

Here, we will make a simple parse that inverts the case of the output from the model.

For example, if the model outputs: "Meow", the parser will produce "mEOW".

In [ ]:
from typing import Iterable

# from langchain_anthropic.chat_models import ChatAnthropic
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage

# model = ChatAnthropic(model_name="claude-2.1")
model = ChatOpenAI(model_name="gpt-3.5-turbo")
# model = ChatOpenAI(model_name="gpt-4o")


def parse(ai_message: AIMessage) -> str:
    """Parse the AI message."""
    return ai_message.content.swapcase()


chain = model | parse
chain.invoke("hello")

### tip
LCEL automatically upgrades the function parse to RunnableLambda(parse) when composed using a | syntax.

If you don't like that you can manually import RunnableLambda and then runparse = RunnableLambda(parse).

* Does streaming work?

In [ ]:
for chunk in chain.stream("Hello"):
    print(chunk, end="|", flush=True)

No, it doesn't because the parser aggregates the input before parsing the output.

If we want to implement a streaming parser, we can have the parser accept an iterable over the input instead and yield the results as they're available.

### Stream parser

In [ ]:
from langchain_core.runnables import RunnableGenerator
from langchain_core.messages import AIMessageChunk

def streaming_parse(chunks: Iterable[AIMessageChunk]) -> Iterable[str]:
    for chunk in chunks:
        yield chunk.content.swapcase()

streaming_parse = RunnableGenerator(streaming_parse)

### tip
Please wrap the streaming parser in RunnableGenerator as we may stop automatically upgrading it with the | syntax.

In [ ]:
chain = model | streaming_parse
chain.invoke("hello")

In [ ]:
for chunk in chain.stream("tell me about yourself in one sentence"):
    print(chunk, end="|", flush=True)

## Inherting from Parsing Base Classes